In [1]:
nums = list(range(1, 11))

### Map, filter and accumulate

In [2]:
squares = list(map(lambda x: x**2, nums))
print(*squares)

1 4 9 16 25 36 49 64 81 100


In [3]:
from operator import add, mul, methodcaller as method

numsPlusSquares = list(map(add, nums, squares))
print(*numsPlusSquares)

2 6 12 20 30 42 56 72 90 110


In [4]:
evens = list(filter(lambda x: not x % 2, nums))
print(*evens)

2 4 6 8 10


In [5]:
from itertools import accumulate

cumsum = accumulate(nums)
cumprod = accumulate(nums, mul)
numsCopy = accumulate(nums, lambda a, b: b)

print(f'{list(cumsum)}\n{list(cumprod)}\n{list(numsCopy)}')

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]
[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


### List folding with reduce

In [6]:
from functools import reduce

total = reduce(add, nums, 0)
print(total)

55


### Examples

Computing factorial:

In [7]:
product = lambda seq: reduce(mul, seq, 1)
ints = lambda n: range(1, n + 1)
ffactorial2 = lambda n: product(ints(n))

In [8]:
print(ffactorial2(5))

120


Dot product function for two vectors stored as Python lists:

In [9]:
dot_prod = lambda u, v: sum(map(lambda t: t[0] * t[1], zip(u, v)))

In [10]:
print(dot_prod([2, 3, 4], [4, 3, 2]))

25


To make things more clear we can define it as the composition of sum of the list and zipping two lists with multuply function.

In [11]:
from toolz import compose
from functools import partial

multuply = lambda t: t[0] * t[1]
zipWith = lambda tfunc: compose(partial(map, tfunc), zip)
dot_prod2 = compose(sum, zipWith(multuply))

In [12]:
print(dot_prod2([2, 3, 4], [4, 3, 2]))

25


Suppose we have this text formatted like a CSV file containing a table where each row except for the header contains information about a person and we want to make a list of dictionaries where each one represents a person with header elements as keys and the corresponding table values as values.

In [13]:
csvFile = \
'''firstName;LastName;Birth;Death
Alonzo;Church;14.06.1903;11.07.1995
Haskell;Curry;12.09.1900;01.09.1982
John;Neumann;28.12.1903;08.02.1957
Alan;Turing;23.06.1912;07.06.1954'''

In [14]:
split = partial(method, 'split')
splitLines = split('\n')
splitFields = split(';')

In [15]:
table = list(map(splitFields, splitLines(csvFile)))
print(*table, sep='\n')

['firstName', 'LastName', 'Birth', 'Death']
['Alonzo', 'Church', '14.06.1903', '11.07.1995']
['Haskell', 'Curry', '12.09.1900', '01.09.1982']
['John', 'Neumann', '28.12.1903', '08.02.1957']
['Alan', 'Turing', '23.06.1912', '07.06.1954']


In [16]:
header, *data = table

In [17]:
makeRecord = partial(compose(dict, zip), header)

In [18]:
records = list(map(makeRecord, data))

In [19]:
for record in records:
    print(record)

{'firstName': 'Alonzo', 'LastName': 'Church', 'Birth': '14.06.1903', 'Death': '11.07.1995'}
{'firstName': 'Haskell', 'LastName': 'Curry', 'Birth': '12.09.1900', 'Death': '01.09.1982'}
{'firstName': 'John', 'LastName': 'Neumann', 'Birth': '28.12.1903', 'Death': '08.02.1957'}
{'firstName': 'Alan', 'LastName': 'Turing', 'Birth': '23.06.1912', 'Death': '07.06.1954'}


### The itertools toolbox

In [20]:
from itertools import chain, compress, dropwhile, takewhile, starmap, groupby

`chain` function takes a sequence of iterables and makes a chain out of them. Like the concatenation function:

In [21]:
day1 = [112.35, 813.21, 34.55, 891.44]
day2 = [233.37,  761.09, 871.59,  725.84]
day3 = [41.81, 67.65, 109.46, 177.11]

allDays = chain(day1, day2, day3)
print(*allDays)

112.35 813.21 34.55 891.44 233.37 761.09 871.59 725.84 41.81 67.65 109.46 177.11


`concat` takes an iterable that consists of iterables and constructs one iterable from these inner ones.

In [22]:
concat = lambda seq: chain(*seq)

`concatMap` just maps over an iterable and applies concat to the result.

In [23]:
concatMap = compose(concat, map)

`compress` is a function that takes two iterables one of which is considered to be a mask: an iterable containing boolean values. It returns the result of applying this mask to the second given iterable, that is, filtering out all the elements that correspond to `False` in the mask.

In [24]:
mask = [0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1]
data = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5, 8, 9, 7, 9, 3, 2, 3, 8, 4, 6, 2, 6, 4, 3, 3, 8, 3, 2, 7, 9, 5]

filtered = compress(data, mask)

print(list(filtered))

[1, 4, 1, 2, 6, 3, 5, 8, 3, 8, 4, 3, 8, 2, 7, 5]


`takewhile` and `dropwhile` functions take a predicate and an iterable and then they are only taking (or dropping) the elements of the iterable while the predicate holds. Once the condition is broken for the first time, the function stops taking values and all the values taken until that moment are the result of the function.

In [25]:
odd = lambda x: x % 2

nums = [2, 7, 1, 8, 2, 8, 1, 8, 2, 8, 4, 5, 9]
nums2 = list(takewhile(odd, nums))
nums3 = list(dropwhile(odd, nums))

print(f'{nums2}\n{nums3}')

[]
[2, 7, 1, 8, 2, 8, 1, 8, 2, 8, 4, 5, 9]


`starmap` is like `map` except it doesn't take a number of sequences to take arguments for a given function from, but instead it maps over a sequence of tuples, takes each tuple, unpacks it to get whatever is inside a tuple, and then uses unpacked values as arguments for the given function.

In [26]:
quotes = {  
'Poincare' : '''The essential characteristic of reasoning by recurrence is that it contains,
condensed, so to speak, in a single formula, an infinity of syllogisms''',

'Turing' : '''We can only see a short distance ahead,
but we can see plenty there that needs to be done''',

'Gauss' : '''God does arithmetic'''
}

In [27]:
newQuotes = starmap(lambda n, q: q + ' -- ' + n, list(quotes.items()))

for q in list(newQuotes):
    print(q)

The essential characteristic of reasoning by recurrence is that it contains,
condensed, so to speak, in a single formula, an infinity of syllogisms -- Poincare
We can only see a short distance ahead,
but we can see plenty there that needs to be done -- Turing
God does arithmetic -- Gauss


`groupby` takes two arguments: a sequence of elements we wish to group and a feature we want them to be grouped by. Returns a list of tuples where the first element is the value of the feature function defining a group, and the second is the list of all the elements of the original sequence that belong to this group in terms of value of the feature function.

In [28]:
vars = ['foo', 'bar', 'baz', 'qux', 'quux', 'corge', 'grault', 'garply']
groups = groupby(vars, len)

for (l, g) in groups:
    print(l, *g)

3 foo bar baz qux
4 quux
5 corge
6 grault garply


In [29]:
from itertools import product as cartProduct

`cartProduct` - the Cartesian product of the given iterables.

In [30]:
rolls = cartProduct(ints(6), repeat=2)

for outcome in rolls:
    print(outcome)

(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
(4, 5)
(4, 6)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 5)
(5, 6)
(6, 1)
(6, 2)
(6, 3)
(6, 4)
(6, 5)
(6, 6)


### One more example
Permutations:

In [31]:
from operator import ne as neq

def tail(a):
    return a[1:]

def remove(e, seq):
    p = partial(neq, e)
    
    return list(chain(
        takewhile(p, seq),
        tail(list(dropwhile(p, seq)))
    ))

def perms(lst):
    mapfunc = lambda x: map(partial(add, [x]), perms(remove(x, lst)))
    
    if len(lst) == 1:
        return [lst]
    else:
        return concatMap(mapfunc, lst)
    
list(perms([1, 2, 3]))

[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]]